# **Householder Notebook**

@eti-calde

---

In [37]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown

## **1.1 Householder Reflection**

**Definition**
Given a unit vector $u$ defining a hyperplane perpendicular to ir, a Householder reflection over a vector $x$ is defined as:

$$
H_{\mathbf{u}}(x) = x-2 \langle \mathbf{x},\mathbf{u}\rangle \mathbf{u}
$$

Is nothing more that the vector reflected over the hyperplane. This can be expressed in matrix form as follows:

$$
H =(I-2\mathbf{u}^T\mathbf{u})
$$

---

## **1.2 let’s see the transformation in code!!**

In [38]:
def get_householder_matrix(v):
    """
    Calculates the Householder reflection matrix for a vector v
    
    Parameters
    ----------
    v : ndarray (n,), input vector.
    Returns
    -------
    H : ndarray (n,n), Householder reflection matrix.
    """
    v = v.reshape(-1, 1)
    I = np.eye(v.shape[0])
    return I - 2 * (v @ v.T) / (v.T @ v)

In [39]:
cmap = plt.get_cmap("magma")
C_V1   = cmap(0.15)  # x
C_V2   = cmap(0.35)  # u
C_PLN  = cmap(0.85)  # hyperplane
C_REF  = cmap(0.60)  # H(x)

def plot_householder(theta1_deg=30.0, theta2_deg=120.0, r1=1.5):
    # Parameters
    t1 = np.deg2rad(theta1_deg)
    t2 = np.deg2rad(theta2_deg)
    unitari_rad = 1.0

    # Vectors
    x = np.array([r1*np.cos(t1), r1*np.sin(t1)])
    u = np.array([unitari_rad*np.cos(t2), unitari_rad*np.sin(t2)])

    # Hyperplane
    n_perp = np.array([-u[1], u[0]])

    # Householder
    H = get_householder_matrix(u)
    x_householder = H @ x
    print("x:", x)
    print("x' (Householder):", x_householder)
    # ===================================================================
    # PLOT
    plt.figure()

    # Vectors
    plt.quiver(0, 0, x[0], x[1], color=C_V1, scale=1, scale_units='xy', angles='xy', label='x')
    plt.quiver(0, 0, u[0], u[1], color=C_V2, scale=1, scale_units='xy', angles='xy', label='u (unitario)')
    plt.quiver(0, 0, x_householder[0], x_householder[1],
               color=C_REF, scale=1, scale_units='xy', angles='xy', label='H(x)')
    # Hyperplane
    L = 2.2
    pts = np.vstack([-L*n_perp, L*n_perp])
    plt.plot(pts[:,0], pts[:,1], linestyle='--', color=C_PLN, label='Hip. u·x = 0')

    # Details
    lim = 2.2
    plt.xlim(-lim, lim); plt.ylim(-lim, lim)
    plt.grid(True)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend()
    plt.title("Householder reflection of x")
    plt.show()

angle1 = FloatSlider(value=30.0,  min=0.0, max=360.0, step=1.0, description='Ángulo x (°)')
angle2 = FloatSlider(value=120.0, min=0.0, max=360.0, step=1.0, description='Ángulo u (°)')
len_r1 = FloatSlider(value=1.5,   min=0.0, max=2.0,   step=0.1, description='||x||')

interact(plot_householder, theta1_deg=angle1, theta2_deg=angle2, r1=len_r1)


interactive(children=(FloatSlider(value=30.0, description='Ángulo x (°)', max=360.0, step=1.0), FloatSlider(va…

<function __main__.plot_householder(theta1_deg=30.0, theta2_deg=120.0, r1=1.5)>

Now let's apply the Householder Reflection to get the QR factorization of a matrix $A$.

---

## **2.3 QR Factorization**

In [40]:
def get_Q_Matrix(householder_vectors):
    """
    Calculates Q matrix of QR factorization using Householder reflections.

    Parameters
    ----------
    householder_vectors : ndarray (n,n), Householder reflection vectors.
    each i-th column represents a Householder vector for the i-th step of the QR factorization.

    Returns
    -------
    Q : ndarray (n,n), Orthogonal matrix.
    """
    n = householder_vectors.shape[0]
    Q = np.eye(n)
    for i in range(n):
        u = householder_vectors[:n-i, i]
        H_tilde = get_householder_matrix(u)
        H_full = np.eye(n)
        H_full[i:, i:] = H_tilde
        Q = Q @ H_full
    return Q

def QR_Householder(A):
    """
    QR factorization using Householder reflections.

    Parameters
    ----------
    A : ndarray (n,n), input matrix.
    Returns
    -------
    Q,R : ndarray (n,n), Orthogonal matrix and upper triangular matrix.
    """
    if (A.shape[1] != A.shape[0]):
        print("Matrix is not square")
        return None, None
    
    n = A.shape[0]
    # We need to work with submatrices of A
    # and we don't want to modify the original Matrix
    A = A.copy()

    # Each Householder vector is 1 dimension smaller than the previous one,
    # So we use just the first n-i rows on each iteration 
    householder_vectors = np.zeros((n, n))

    for i in range(n):
        sub_A = A[i:, i:]
        # Get Column i of A
        col_i_of_A = sub_A[:,0]
        col_i_of_A = col_i_of_A.reshape(-1,1)

        # Compute the Householder vector
        alpha = -np.sign(col_i_of_A[0,0]) * np.linalg.norm(col_i_of_A)
        e1 = np.eye(col_i_of_A.shape[0])[:,0].reshape(-1,1)
        u = col_i_of_A - alpha * e1
        u = u / np.linalg.norm(u)
        # print(f"u en la iteracion {i}, {u.shape} =\n", u)

        # Save the Householder vector
        householder_vectors[0:n-i,i] = u.ravel()
        # print(f"holder_vectors en la iteracion {i} =\n", holder_vectors)
        
        # FIX RESPECTO A LA VERSIÓN DE LA CALSE ANTERIOR.
        A[i:, i:] -= 2 * np.outer(u, u.T @ A[i:, i:])

    Q = get_Q_Matrix(householder_vectors)
    R = np.triu(A)
    # R = A
    return Q,R

In [41]:
size = 4
A = np.random.rand(size,size) + np.identity(size)*2
print("Matriz A\n",A)
Q,R = QR_Householder(A)
# print("uni?:\n", Q.T@Q)
compare_numpy = True
if (size <= 5):
    # print("A =\n", A)
    print("R:\n", R)
    print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A - Q @ R))
    if compare_numpy:
        A_np, Q_np, R_np = A.copy(), Q.copy(), R.copy()
        print("Comparando con numpy...")
        Q_np, R_np = np.linalg.qr(A_np)
        print("R (numpy):\n", R_np)
        print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A_np - Q_np @ R_np))
else:
    print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A - Q @ R))
    if compare_numpy:
        A_np, Q_np, R_np = A.copy(), Q.copy(), R.copy()
        print("Comparando con numpy...")
        Q_np, R_np = np.linalg.qr(A_np)
        print("Reconstrucción ||A - Q R|| =", np.linalg.norm(A_np - Q_np @ R_np))

Matriz A
 [[2.6075 0.1705 0.0651 0.9489]
 [0.9656 2.8084 0.3046 0.0977]
 [0.6842 0.4402 2.122  0.4952]
 [0.0344 0.9093 0.2588 2.6625]]
R:
 [[-2.8638 -1.2183 -0.6721 -1.0472]
 [ 0.     -2.7299 -0.4458 -0.6591]
 [ 0.      0.     -2.0041 -0.4159]
 [ 0.      0.      0.     -2.5574]]
Reconstrucción ||A - Q R|| = 1.746024815751244e-15
Comparando con numpy...
R (numpy):
 [[-2.8638 -1.2183 -0.6721 -1.0472]
 [ 0.     -2.7299 -0.4458 -0.6591]
 [ 0.      0.     -2.0041 -0.4159]
 [ 0.      0.      0.      2.5574]]
Reconstrucción ||A - Q R|| = 1.1124109571185501e-15


---
## **2.4 Arnoldi Householder**

### Idea Principal

El resultado del algoritmo de Arnoldi se basa en la reducción parcial de $A$ a una forma Hesenberg:
$$
A V_k = V_{k+1} \tilde{H}_k \tag{1}
$$

Recordemos que $V_k$ es el espacio que deseamos ortonormalizar qu esta compuesto por los vectores del sub espacio de Krylov:
$$
V_k = \{r_0, A r_0, A^2 r_0, \dots, A^{k-1}r_0\}
$$

Ahora notamos que el lado izquierdo de la ecuación **(1)** $A V_k$ podemos escribir la multiplicación así:
$$
A V_k = \{Ar_0, A^2  r_0, A^3 r_0, \dots, A^{k}r_0\}
$$

Lo cual es muy cercano a $V_{k+1}$, pero nos faltaría agregar la primera columna al espacio que es $r_0$, llegando a:
$$
[r_0 A V_k] = V_{k+1} R_k \tag{2}
$$

Donde:
$$
R_k =
\begin{bmatrix}
\beta  & h_{1,1} & \dots  & h_{1,k} \\
0      & h_{2,1} & \dots  & h_{2,k} \\
\vdots & \vdots & \ddots & \vdots \\
0      & 0      & \dots  & h_{k+1,k}
\end{bmatrix}
$$

El objetivo de hacer esto es que ahora en la ecucación **(2)** tenemos una matriz del lado izquierdo $[r_0 A V_k]$ y del lado derecho una matriz ortogonal $V_{k+1}$ y una matriz triangular superior $R_k$ de decir podemos llegar al lado derecho aplicando QR al lado izquierdo.

### Detalles

Para aplicar Arnoldi en GMRes no se parte de la base completa $V_k$, mas bien se construye en cada iteración un nuevo vector de la base, por lo cual surge la duda ¿cómo se obtiene el vector $v_i$?

1. El plan es hacer factorización QR sobre $[r_0 A V_k]$:
$$
[r_0 A V_k] = Q R
$$

donde $Q = V_{k+1}$ porque la construcción viene de la reducción parcial de Hessenberg y por lo aprendido en QR Householder, sabemos que:
$$
V_{k+1} = Q = P_1 P_2 \dots P_{k+1}
$$

Las columnas de la matriz $V_{k+1}$ son los $v_i$ que andamos buscando y ahora es sencillo que que si queremos obenter la i-ésima columna podemos:

$$
v_i = V_{k+1} e_i = (P1 P2 \dots P_{k+1}) e_i
$$

Es más aún, dada la estructura de las matrices $P$, recordad que cada una es una identidad con una submatriz dentro, por lo tanto:

$$
v_i = P_1 P_2 \dots P_j e_j
$$





In [42]:
size = 5
A = np.random.rand(size, size)
e3 = np.zeros(size)
e3[2] = 1
print(A)
print("\n",A @ e3)

[[0.3117 0.5201 0.5467 0.1849 0.9696]
 [0.7751 0.9395 0.8948 0.5979 0.9219]
 [0.0885 0.196  0.0452 0.3253 0.3887]
 [0.2713 0.8287 0.3568 0.2809 0.5427]
 [0.1409 0.8022 0.0746 0.9869 0.7722]]

 [0.5467 0.8948 0.0452 0.3568 0.0746]


In [43]:
def get_householder_vector(x):
    if np.linalg.norm(x) < 1e-16:
        return np.zeros_like(x)
    
    alpha = np.linalg.norm(x)
    e1 = np.zeros_like(x)
    e1[0] = 1.0
    
    # El signo garantiza la estabilidad numérica evitando la resta de números cercanos
    v = x + np.copysign(alpha, x[0]) * e1
    v /= np.linalg.norm(v)
    return v

# =============== TEST ===============

# rand_vec = np.random.rand(3)
# norm = np.linalg.norm(rand_vec)
# householder_vec = get_householder_vector(rand_vec)
# apply_householder = rand_vec - 2 * householder_vec * (householder_vec.T @ rand_vec)
# e1 = np.zeros(3)
# e1[0] = 1
# print("original vec:\n", rand_vec)
# print("apply householder vec:\n", apply_householder)
# print("|rand vec| * e1:\n", norm * e1)

# =====================================


In [44]:
def apply_householder_sequence(x, householder_vecs, reverse=False):
    vec = x.copy()
    iterator = reversed(householder_vecs) if reverse else householder_vecs
    for w in iterator:
        vec = vec - 2 * w * (w.T @ vec)     
    return vec

In [45]:
def householder_arnoldi(A, r0, m, verbose=False):
    debug = lambda msg: print(msg)  if verbose else None

    n = A.shape[0]
    V = np.zeros((n, m + 1))
    H_bar = np.zeros((m + 1, m))
    householder_vecs = []
    z = r0.copy() # Vectors to transform to form H_bar

    rhs_scalar = 0.0

    for k in range(m+1):
        debug(f"\n =============== ITERARTION j={k} =============== ")
        debug(f"\n =================== STEP 1 =================== ")     
        debug(f"1. Get householder vector w_{k} hat send z to |z|e1")    
        
        sub_z       = z[k:]
        v           = get_householder_vector(sub_z)        
        w_k         = np.zeros(n)
        w_k[k:] = v
        householder_vecs.append(w_k)
        
        debug(f"2. Householder Vector w_{k}:\n {w_k}")                    
        debug(f"\n ================================================ \n")  

        debug(f"\n =================== STEP 2 =================== ")      
        debug("1. Apply the householder transformation w_{j} to z ")      
        debug("We do not update H_bar on the first iteration") if k==0 else None

        h_full = z - 2 * w_k * (w_k.T @ z)
        if k > 0:
            H_bar[:, k - 1] = h_full[:m + 1]
            debug(f"2. H_bar updated (column {k - 1}):\n {H_bar}\n")   
        elif k ==0:
            rhs_scalar = h_full[0]
        debug(f"\n ================================================ \n")  
        
        debug(f"\n =================== STEP 3 =================== ")      
        debug("1. Construct the new vector of V applying the householder sequence")   
        e_k = np.zeros(n)
        e_k[k] = 1.0
        v_k = apply_householder_sequence(e_k, householder_vecs, reverse=True)
        V[:, k] = v_k
        
        debug(f"2. new v_{k} = \n{v_k}")
        debug(f"3. add v_{k} a V[:, {k}] \n{V}")
        debug(f"\n ================================================ \n")


        if k <= m:
            debug(f"\n =================== STEP 4 =================== ")   
            debug("1. Prepare z for the next iteration")
            Av = A @ v_k
            z = apply_householder_sequence(Av, householder_vecs, reverse=False)
            debug(f"new vector z_{k+1} = A v_{k} and then apply old reflections  = \n{z}\n")
            debug(f"\n ================================================ \n")

    V_m = V[:, :m]
    V_m_plus_1 = V

    if verbose:
        debug(f"\n =================== EXTRA CHECKS =================== ")
        debug("1. Check the orthonormality of V_m: ")
        debug(f"| I - V_m^T @ V_m | :\n {np.linalg.norm(np.identity(m)- (V_m.T @ V_m))}")
        LHS = A @ V_m
        RHS = V_m_plus_1 @ H_bar
        error = np.linalg.norm(LHS - RHS)
        debug("2. Verify Arnoldi's fundamental identity ")
        debug(f"\nError ||A*V_m - V_{{m+1}}*H_bar||: {error:.2e}")

    return V_m, V_m_plus_1, H_bar, rhs_scalar


np.random.seed(0)
n = 5
A = np.random.rand(n, n)
r0 = np.random.rand(n)
r0 = r0 / np.linalg.norm(r0)
m = 4

V_m, V_m_plus_1, H_bar, _ = householder_arnoldi(A, r0, m, verbose=True)




 =============== ITERARTION j=0 =============== 

 =================== STEP 1 =================== 
1. Get householder vector w_0 hat send z to |z|e1
2. Householder Vector w_0:
 [0.8607 0.0627 0.4129 0.2281 0.1812]



 =================== STEP 2 =================== 
1. Apply the householder transformation w_{j} to z 
We do not update H_bar on the first iteration



 =================== STEP 3 =================== 
1. Construct the new vector of V applying the householder sequence
2. new v_0 = 
[-0.4814 -0.1079 -0.7107 -0.3926 -0.312 ]
3. add v_0 a V[:, 0] 
[[-0.4814  0.      0.      0.      0.    ]
 [-0.1079  0.      0.      0.      0.    ]
 [-0.7107  0.      0.      0.      0.    ]
 [-0.3926  0.      0.      0.      0.    ]
 [-0.312   0.      0.      0.      0.    ]]



 =================== STEP 4 =================== 
1. Prepare z for the next iteration
new vector z_1 = A v_0 and then apply old reflections  = 
[ 2.4298 -1.2318  0.4735 -0.2732 -0.482 ]




 =============== ITERARTION j=

# GMRes Householder LSTQ

In [46]:
def gmres_householder_lstsq(A, b, m, tol=1e-10, verbose=False):
    n = A.shape[0]
    
    # --- INICIALIZACIÓN DE GMRES ---
    x0 = np.zeros(n)
    r0 = b - A @ x0
    beta = np.linalg.norm(r0)
    r0 = r0 / beta
    
    residuals_history = [beta]
    
    if beta < tol:
        return x0, residuals_history

    V_m, V_m_plus_1, H_bar, rhs_escalar = householder_arnoldi(A, r0, m, verbose=False)
    print(rhs_escalar)
        
    # 1. Definir el lado derecho del sistema de mínimos cuadrados
    # DETALLE DE VITAL IMPORTANCIA Y DONDE PERDÍ HORASSSS :c
    # Householder no manda v1 (v1 = r0/beta) a (||v1|| e1)
    # Puede ser a (||v1|| * +e1) o (||v1|| * -e1)
    rhs = np.zeros(m + 1)
    rhs[0] = rhs_escalar * beta
    
    # 2. Resolver el sistema para encontrar y
    y, _, _, _ = np.linalg.lstsq(H_bar, rhs, rcond=None)
    
    # 3. Calcular la solución final actualizando x0
    x_final = x0 + V_m @ y
    
    # Cálculo del historial de residuos para análisis
    for k in range(1, m):
        yk, _, _, _ = np.linalg.lstsq(H_bar[:k+1, :k], rhs[:k+1], rcond=None)
        xk = x0 + V_m_plus_1[:, :k] @ yk
        residuals_history.append(np.linalg.norm(b - A @ xk))
    
    residuals_history.append(np.linalg.norm(b - A @ x_final))

    return x_final, residuals_history

np.random.seed(42)
n = 10
A = np.random.rand(n, n) + np.identity(n) * 2
x = np.ones(n)
b = A @ x
m = 8

x_sol, res_hist = gmres_householder_lstsq(A, b, m)

final_residual_norm = np.linalg.norm(b - A @ x_sol)
print(f"Solución encontrada: {x_sol}")
print("\nHistorial de la norma del residuo (calculado a posteriori):")
for i, res in enumerate(res_hist):
    print(f"  Iteración {i}: {res:.4e}")

-1.0000000000000004
Solución encontrada: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]

Historial de la norma del residuo (calculado a posteriori):
  Iteración 0: 2.1272e+01
  Iteración 1: 6.4041e-01
  Iteración 2: 2.5845e-01
  Iteración 3: 1.0048e-01
  Iteración 4: 2.3385e-02
  Iteración 5: 5.4823e-03
  Iteración 6: 1.1037e-03
  Iteración 7: 2.6706e-04
  Iteración 8: 3.4848e-05


---

# Gmres Householder Givens\#2


In [47]:
def givens_rotation(a,b):
    """
    Calcula los valores de seno y coseno para una rotación de Givens.
    """
    if b == 0:
        c = 1.0
        s = 0.0
    else:
        hyp = np.hypot(a, b)
        c = a / hyp
        s = -b / hyp
    return c, s

In [51]:
def gmres_householder_givens(A, b, m, verbose=False, tol = 1e-10):
    debug = lambda msg: print(msg)  if verbose else None

    n = A.shape[0]
    V = np.zeros((n, m + 1))
    H_bar = np.zeros((m + 1, m))
    householder_vecs = []
    x0 = np.zeros(n)
    r0 = b - A @ x0
    beta = np.linalg.norm(r0)
    z = r0.copy()/beta # Vectors to transform to form H_bar

    residuals_history = [beta]
    
    if beta < tol:
        return x0, residuals_history
    
    rhs_scalar = 0.0
    rhs = np.zeros(m + 1)

    for k in range(m+1):
        debug(f"\n =============== ITERARTION j={k} =============== ")
        debug(f"\n =================== STEP 1 =================== ")     
        debug(f"1. Get householder vector w_{k} hat send z to |z|e1")    
        
        sub_z       = z[k:]
        v           = get_householder_vector(sub_z)        
        w_k         = np.zeros(n)
        w_k[k:] = v
        householder_vecs.append(w_k)
        
        debug(f"2. Householder Vector w_{k}:\n {w_k}")                    
        debug(f"\n ================================================ \n")  

        debug(f"\n =================== STEP 2 =================== ")      
        debug("1. Apply the householder transformation w_{j} to z ")      
        debug("We do not update H_bar on the first iteration") if k==0 else None

        h_full = z - 2 * w_k * (w_k.T @ z)
        if k > 0:
            H_bar[:, k - 1] = h_full[:m + 1]
            debug(f"2. H_bar updated (column {k - 1}):\n {H_bar}\n")   
        elif k ==0:
            rhs_scalar = h_full[0]
            rhs[0] = rhs_scalar * beta
        debug(f"\n ================================================ \n")  
        
        debug(f"\n =================== STEP 3 =================== ")      
        debug("1. Construct the new vector of V applying the householder sequence")   
        e_k = np.zeros(n)
        e_k[k] = 1.0
        v_k = apply_householder_sequence(e_k, householder_vecs, reverse=True)
        V[:, k] = v_k
        
        debug(f"2. new v_{k} = \n{v_k}")
        debug(f"3. add v_{k} a V[:, {k}] \n{V}")
        debug(f"\n ================================================ \n")


        if k <= m:
            debug(f"\n =================== STEP 4 =================== ")   
            debug("1. Prepare z for the next iteration")
            Av = A @ v_k
            z = apply_householder_sequence(Av, householder_vecs, reverse=False)
            debug(f"new vector z_{k+1} = A v_{k} and then apply old reflections  = \n{z}\n")
            debug(f"\n ================================================ \n")


        if k > 0:
            debug(f"\n =================== STEP 5 =================== ")
            debug(f"1. Get the c,s values that define the Given Rotation to triangularize the column {k-1}")
            c, s = givens_rotation(H_bar[k-1,k-1], H_bar[k,k-1])
            
            H_row_k_minus_1 = c * H_bar[k-1, :] - s * H_bar[k-1, :]
            H_row_k = s * H_bar[k, :] + c * H_bar[k, :]
            H_bar[k-1, :] = H_row_k_minus_1
            H_bar[k, :] = H_row_k
            debug(f"2. Apply the rotation to H_bar:\n {H_bar}")
            
            rhs_row_k_minus_1 = c * rhs[k-1, :] - s * rhs[k-1, :]
            rhs_row_k = s * rhs[k, :] + c * rhs[k, :]
            rhs[k-1, :] = rhs_row_k
            H_bar[k, :] = res_hist
            debug(f"3. Apply the rotation to rhs vector: \n, this is the original :\n {rhs}")
            debug("And this is the vector after apply the rotation: \n {rhs}")
            debug(f"\n ================================================ \n")


            debug(f"\n =================== STEP 6 =================== ")
            debug(f"Now we can solve the LSTQ problem")
            
            y, _, _, _ = np.linalg.lstsq(H_bar, rhs, rcond=None)
    
            x_k = x0 + V_m @ y
            residuals_history(np.linalg.norm(b - A @ x_k))


    return x_k, residuals_history


np.random.seed(0)
n = 3
A = np.random.rand(n, n)
x_true = np.ones(n)
b = A @ x_true
m = 2

x_k, residuals = gmres_householder_givens(A, b, m, verbose=True)
print(x_true)
print(x_k)


 =============== ITERARTION j=0 =============== 

 =================== STEP 1 =================== 
1. Get householder vector w_0 hat send z to |z|e1
2. Householder Vector w_0:
 [0.8815 0.2718 0.3861]



 =================== STEP 2 =================== 
1. Apply the householder transformation w_{j} to z 
We do not update H_bar on the first iteration



 =================== STEP 3 =================== 
1. Construct the new vector of V applying the householder sequence
2. new v_0 = 
[-0.5541 -0.4792 -0.6807]
3. add v_0 a V[:, 0] 
[[-0.5541  0.      0.    ]
 [-0.4792  0.      0.    ]
 [-0.6807  0.      0.    ]]



 =================== STEP 4 =================== 
1. Prepare z for the next iteration
new vector z_1 = A v_0 and then apply old reflections  = 
[ 1.9408 -0.0202 -0.0128]




 =============== ITERARTION j=1 =============== 

 =================== STEP 1 =================== 
1. Get householder vector w_1 hat send z to |z|e1
2. Householder Vector w_1:
 [ 0.     -0.9605 -0.2784]



 ===

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

# GMRES GIven GEMINI

In [49]:
import numpy as np
from scipy.linalg import hessenberg

# Tus funciones auxiliares (givens_rotation, etc.) están bien.
# Las incluyo aquí para que el código sea autoejecutable.
def givens_rotation(a, b):
    if b == 0:
        c, s = 1.0, 0.0
    else:
        hyp = np.hypot(a, b)
        c = a / hyp
        s = -b / hyp
    return c, s

# Estas son funciones de Householder que tu código original usa pero no incluyó.
# Las agrego para que el ejemplo funcione.
def get_householder_vector(x):
    v = x.copy()
    v[0] += np.sign(x[0]) * np.linalg.norm(x)
    v /= np.linalg.norm(v)
    return v

def apply_householder_sequence(x, vecs, reverse=False):
    y = x.copy()
    sequence = reversed(vecs) if reverse else vecs
    for v in sequence:
        y -= 2 * v * (v.T @ y)
    return y


def gmres_householder_givens(A, b, x0, m, tol=1e-10, verbose=False):
    debug = lambda msg: print(msg) if verbose else None

    # ====================== GMRES INITIALIZATION ======================
    n = A.shape[0]
    r0 = b - A @ x0
    beta = np.linalg.norm(r0)
    
    residuals = [beta]
    if beta < tol:
        return x0, residuals

    # ===================== ARNOLDI INITIALIZATION =====================
    V = np.zeros((n, m + 1))
    H_bar = np.zeros((m + 1, m))
    householder_vecs = []
    
    # El vector z inicial es el residuo normalizado r0
    z = r0 / beta
    
    # ===================== GIVENS INITIALIZATION =====================
    c = np.zeros(m)
    s = np.zeros(m)
    # g es el lado derecho del sistema minimizado. G*H_bar*y = G*(beta*e1)
    g = np.zeros(m + 1)
    g[0] = beta
    
    # El bucle principal es más claro de 0 a m-1
    for k in range(m):
        debug(f"\n =============== ITERATION k={k} =============== ")
        
        # --- PASOS DE HOUSEHOLDER PARA CONSTRUIR H_bar y V ---
        # (Esta parte de tu lógica estaba mayormente bien, la he adaptado al nuevo bucle)
        
        # 1. Obtener el vector de Householder w_{k+1}
        sub_z = z[k:]
        v = get_householder_vector(sub_z)
        w_k = np.zeros(n)
        w_k[k:] = v
        householder_vecs.append(w_k)
        
        # 2. Aplicar la transformación a z para obtener la nueva columna de H_bar
        h_col_full = z - 2 * w_k * (w_k.T @ z)
        H_bar[:, k] = h_col_full[:m + 1]
        
        # 3. Construir el nuevo vector v_{k+1} para la base de Krylov
        e_k = np.zeros(n)
        e_k[k] = 1.0
        v_k = apply_householder_sequence(e_k, householder_vecs[:k+1], reverse=True)
        V[:, k] = v_k
        
        # 4. Preparar z para la siguiente iteración
        Av = A @ v_k
        z = apply_householder_sequence(Av, householder_vecs, reverse=False)

        # ===================== PASOS DE GIVENS (LA PARTE CORREGIDA) =====================
        debug(f"\n =================== STEP 5 (Givens k={k}) =================== ")
        
        # 5.1: Aplicar las rotaciones PREVIAS (k-1) a la NUEVA columna de H_bar
        # Esto pone "al día" a la columna que acabamos de calcular.
        for i in range(k):
            h_ik = H_bar[i, k]
            h_i1k = H_bar[i + 1, k]
            H_bar[i, k]   = c[i] * h_ik - s[i] * h_i1k
            H_bar[i + 1, k] = s[i] * h_ik + c[i] * h_i1k

        # 5.2: Calcular la NUEVA rotación (c_k, s_k) para anular H_bar[k+1, k]
        # Se calcula sobre los elementos ya transformados por las rotaciones anteriores.
        c[k], s[k] = givens_rotation(H_bar[k, k], H_bar[k + 1, k])
        
        # 5.3: Aplicar la NUEVA rotación a la matriz H_bar y al vector g
        # Aplicar a H_bar (solo afecta a la diagonal y crea el cero)
        h_kk = H_bar[k, k]
        H_bar[k, k] = c[k] * h_kk - s[k] * H_bar[k+1, k]
        H_bar[k+1, k] = 0.0 # Se anula por definición de la rotación

        # Aplicar a g
        g_k = g[k]
        g[k]   = c[k] * g_k - s[k] * g[k+1] # g[k+1] es cero antes de este paso
        g[k+1] = s[k] * g_k + c[k] * g[k+1]
        
        # 5.4: La norma del residuo es el valor absoluto del último elemento de g
        residual_norm = abs(g[k+1])
        residuals.append(residual_norm)
        debug(f"H_bar after Givens:\n {H_bar[:k+2, :k+1].round(4)}")
        debug(f"g after Givens: {g[:k+2].round(4)}")
        debug(f"Residual norm at this step: {residual_norm:.4e}")
        
        if residual_norm < tol:
            break

    # ===================== SOLVE AND UPDATE =====================
    # Al final del bucle (o si converge), k es el número de iteraciones realizadas.
    # Necesitamos resolver el sistema triangular superior R*y = g'
    # donde R es H_bar[:k+1, :k+1] y g' es g[:k+1]
    
    R = H_bar[:k+1, :k+1]
    g_final = g[:k+1]
    
    # Usamos un solver para sistemas triangulares (más eficiente que lstsq)
    from scipy.linalg import solve_triangular
    y = solve_triangular(R, g_final, lower=False)
    
    # Actualizamos la solución
    x_final = x0 + V[:, :k+1] @ y
    
    return x_final, residuals


# --- EJEMPLO DE USO ---
np.set_printoptions(precision=4, suppress=True)
np.random.seed(42)
n = 4
A = np.random.rand(n, n) + np.identity(n) * 2
x_true = np.ones(n)
b = A @ x_true
x0 = np.zeros(n)
m = 3 # Máximo de iteraciones

# Ejecutar el solver
x_sol, residuals_history = gmres_householder_givens(A, b, x0, m, tol=1e-8, verbose=False) # Verbose a False para una salida limpia

# Verificar la solución
final_residual_norm = np.linalg.norm(b - A @ x_sol)
print(f"\nSolución encontrada {x_true}")
print("Historial de la norma del residuo:")
for i, res in enumerate(residuals_history):
    print(f"  Iteración {i}: {res:.4e}")


Solución encontrada [1. 1. 1. 1.]
Historial de la norma del residuo:
  Iteración 0: 7.8909e+00
  Iteración 1: 4.3803e-16
